# Jupyter React Components

This notebook discusses the potential creation and use of a React style Notebook Extension. The demo uses the `juno-magic` package as a Proof-of-Concept.

## Concept 

The goal of this work is to create an extensible pattern for utilizing React components in a notebook, but have them driven and built by Python on the backend. 


## Communication 

### Comms

Communication between the back-end (python) and front-end (javascript) is handling via custom jupyter `comms`. These are custom messages that can be used to easily link back-end components to front-end components via the websocket based Jupyter Messaging Protocol. 

<img src="./pyreact1.png" width="400" height="200" />


### Dispatching 

On the javascript side React components are initialized with a single comm (each component opens a new comm) so that they can directly send messages back to python if/when their state changes. When messages are sent over comms from Python a central dispatch and relay events and broadcast them to all created components. This way each React component on the front can trigger central dispatches and force re-renders of other components. Everything becomes connectable via the central dispatch.


## Messaging 

When python components need to communicate with JS components they send messages over the components `comm`. These messages provide a `method` (ie `{'method': 'display'}`) and any additional data or props that the JS may need or want.  

### Display

When the ipython method `display` is called on a Python component the corresponding JS component reacts by rendering itself to the DOM. Javascript components are rendered to either a domId (provided by python via the display call) or to a cell's output subarea (default).     

### Update 

Using an `observer` pattern on properties of the Python based components allows for messages to pass over comms whenever Python objects change. This means that if a property in Python is updated/changed the front-end components can react to those changes by listening for events over the open comms.

When a property on a python component is updated or changed it will send a message with the `update` method. The javascript components listen for the `update` and `dispatch` an event. The dispatched events are used to update the state of React Components and hence force updates on the components when needed.  

### Architecture

The architecture consists of loosely coupled components on the back-end and front-end. The back-end instantiates new components, which opens up new comms, which then instantiates new JS components. 

<img src="./pyreact_arch.png" width="600" height="400" />


## Demo: juno_magic

To showcase a proof-of-concept we've built the pattern using the juno_magic package as a use case. There are 2 components built in the POC: `Status` and `List`. The Status component renders a react component to the `kernel_status_indicator` in the notebook menubar. The List component renders a list of kernels to the cell's output area. 

### Status

The status component shows the ability of a REact component to bind to the central dispatch, update its state, and then re-render itself. In the case of Juno kernels when the remote kernels execution state changes (from idle to busy and back) an update message is sent over the comm with the new status. The component dispatches the update event and the status view updates and re-renders.

Also, the status component is initialized in the `__init__` method of the juno_magic. So when the extension is loaded the status component is rendered and binds itself to "status_update" events.  

In [6]:
%reload_ext autoreload
%autoreload 2
%reload_ext juno_magic

### List

When the juno list method is called it instantiates a new List component in python (and a new comm is opened and JS component created). The list component shows off the bi-directional communication of comms. When a list items button is clicked a JS method is invoked, which triggers a message to be sent to the back-end over the comm. 

The python component reacts by creating a new cell and inserting/executing code (to select the kernel).

In [7]:
%juno list

Successfully connected to wss://juno.timbr.io/wamp/route


In [8]:
%juno select 'GBDx Demo'

No previous subscriptions
Kernel selected [io.timbr.kernel.924e80dd-11f6-42d8-9f26-1202be63d184]


## Updating Status

When code is executed on the remote kernel the execution state changes from "idle" to "busy" and then back. This causes the react component to update its states and re-render what it needs to.

In [9]:
%%juno

import time
time.sleep(3)
print 'hello remote kernel'


hello remote kernel
